# ![.right](https://lh6.ggpht.com/wUrTIhpHPVqw_VPCdhbSiF5TXlBlLyRHdH1gsII_y5NkgYKzbbj7cC2l6AGoVq-JN0U=w100 "MSDS 7331 Data Mining - Project 1")


# Project 3: BEER ME! 

### a recommendation system for those who are thirsty but don't know it yet

---



# Team Members

- Chris Woodard
- Claire Chu
- Nathan Mowat
- Bill Kerneckel

---


# Rubic

- [Business Understanding](#dp1) 

- [Data Understanding 1](#dp2)

- [Data Understanding 2](#me1)

- [Modeling and Evaluation 1](#me2)

- [Modeling and Evaluation 2](#me3)

- [Modeling and Evaluation 3](#me4)

- [Modeling and Evaluation 4](#me5)

- [Deployment](#d)

- [Exceptional Work](#ew)

- [Appendix](#a)

<hr>



# OBJECTIVE: 
We found a comprehensive list of beer reviews from beeradvocate and ratebeer. We want to use these reviews to generate a recommendation system that will suggest potential beers that a user might like based on their previous beer ratings and reviews:
- Establish a solid distance metric to determine the similiarity amongst different users
- Utilize this metric to recommend other beers that a user might like

We hope that the collaborative filtering on a custom recommendation system completed in this notebook will help make beer drinkers thirsty for beers they have not tried yet.

source: https://data.world/socialmediadata/beeradvocate
source: http://nbviewer.jupyter.org/gist/glamp/20a18d52c539b87de2af
source: https://snap.stanford.edu/data/web-BeerAdvocate.html
source: https://github.com/yhat/Beer-Rec-Flask/blob/master/README.md

# Business Understanding

---

<span style="color:red">10 Points - Describe the purpose of the data set you selected (i.e., why was this data collected in the first place?). How will you measure the effectiveness of a good algorithm? Why does your chosen validation method make sense for this specific dataset and the stakeholders needs?</span>

This data was collected from Beeradvocate.com and ratebeer.com. Both websites are used by beer drinkers to not only stay informed about the latest in beer news, but to also track and review the beers each user has tried. As an avid beer drinker myself, these websites are useful to me in that they help me avoid re-ordering beers i've previously tried and did not like. This helps me save money and stomach space when i'm ordering drinks at the bar. 

The dataset we are analyzing has a total of 13 columns including: 
-brewery Id
-brewery name
-review time
-overall review rating
-review of aroma
-review of appears
-username of reviewer
-beer style
-review palate
-review taste
-beer name
-alcohol percentage by volume
-beer ID

A good algorithm will be able to suggest a beer that the user will drink and rate at least a 4 or above in their overall rating (we chose 4 because this is the median review rating). If a user reviews the suggested beer at less than 4, our recommender system is not effective. 

## need help here: Statistically, we will use a ....

# Data Understanding

---

<span style="color:red"> 10 Points - Describe the meaning and type of data (scale, values, etc.) for each attribute in the data file. Verify data quality: Are there missing values? Duplicate data? Outliers? Are those mistakes? How do you deal with these problems?</span>

Insert content here!.......

In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
#load python libaries
import numpy as np
import pandas as pd
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as pl
import seaborn as sns
import glob
import sys
import os
import warnings
import ggplot
import datetime
import time
from datetime import date

In [2]:
%matplotlib inline

In [3]:
#load data file
beerDF = pd.read_csv('beer_reviews.csv') 

#you'll have to change the file path to your working directory


In [4]:
# dropping columns we are not using for the analysis
beerDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
brewery_id            1586614 non-null int64
brewery_name          1586599 non-null object
review_time           1586614 non-null int64
review_overall        1586614 non-null float64
review_aroma          1586614 non-null float64
review_appearance     1586614 non-null float64
review_profilename    1586266 non-null object
beer_style            1586614 non-null object
review_palate         1586614 non-null float64
review_taste          1586614 non-null float64
beer_name             1586614 non-null object
beer_abv              1518829 non-null float64
beer_beerid           1586614 non-null int64
dtypes: float64(6), int64(3), object(4)
memory usage: 157.4+ MB


In [5]:
beerDF.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [6]:
summary = beerDF.describe()
summary = summary.transpose()
summary

,count,mean,std,min,25%,50%,75%,max
brewery_id,1586614.0,3.130099e+03,5.578104e+03,1.000000e+00,1.430000e+02,4.290000e+02,2.372000e+03,2.800300e+04
review_time,1586614.0,1.224089e+09,7.654427e+07,8.406720e+08,1.173224e+09,1.239203e+09,1.288568e+09,1.326285e+09
review_overall,1586614.0,3.815581e+00,7.206219e-01,0.000000e+00,3.500000e+00,4.000000e+00,4.500000e+00,5.000000e+00
review_aroma,1586614.0,3.735636e+00,6.976167e-01,1.000000e+00,3.500000e+00,4.000000e+00,4.000000e+00,5.000000e+00
review_appearance,1586614.0,3.841642e+00,6.160928e-01,0.000000e+00,3.500000e+00,4.000000e+00,4.000000e+00,5.000000e+00
review_palate,1586614.0,3.743701e+00,6.822184e-01,1.000000e+00,3.500000e+00,4.000000e+00,4.000000e+00,5.000000e+00
review_taste,1586614.0,3.792860e+00,7.319696e-01,1.000000e+00,3.500000e+00,4.000000e+00,4.500000e+00,5.000000e+00
beer_abv,1518829.0,7.042387e+00,2.322526e+00,1.000000e-02,5.200000e+00,6.500000e+00,8.500000e+00,5.770000e+01
beer_beerid,1586614.0,2.171279e+04,2.181834e+04,3.000000e+00,1.717000e+03,1.390600e+04,3.944100e+04,7.731700e+04


In [7]:
#what is the most popular beer?
beerDF.beer_name.describe()

count           1586614
unique            56857
top       90 Minute IPA
freq               3290
Name: beer_name, dtype: object

In [8]:
#what are the top 10 most popular beers?
from collections import Counter
topbeer = Counter(beerDF.beer_name)
topbeer.most_common(10)

[('90 Minute IPA', 3290),
 ('India Pale Ale', 3130),
 ('Old Rasputin Russian Imperial Stout', 3111),
 ('Sierra Nevada Celebration Ale', 3000),
 ('Two Hearted Ale', 2728),
 ('Arrogant Bastard Ale', 2704),
 ('Stone Ruination IPA', 2704),
 ('Sierra Nevada Pale Ale', 2587),
 ('Stone IPA (India Pale Ale)', 2575),
 ('Pliny The Elder', 2527)]

In [9]:
#use the top three most reviewed beers to see if there are any reviewers that have written reviews on all three
beer_1, beer_2, beer_3 = "90 Minute IPA", "India Pale Ale", "Old Rasputin Russian Imperial Stout"

beer_1_reviewers = beerDF[beerDF.beer_name==beer_1].review_profilename.unique()
beer_2_reviewers = beerDF[beerDF.beer_name==beer_2].review_profilename.unique()
beer_3_reviewers = beerDF[beerDF.beer_name==beer_3].review_profilename.unique()
common_reviewers = set(beer_1_reviewers) & set(beer_2_reviewers) & set(beer_3_reviewers)

print ("Users in the same set: %d " % len(common_reviewers))
list(common_reviewers)[:20]


Users in the same set: 809 


['Px2GrafX',
 'wvsabbath',
 'scottfrie',
 'carln26',
 'aasher',
 'ElGuapo',
 'titosupertramp',
 'GeoffRizzo',
 'stephens101',
 'jmalex',
 'mikeg67',
 'projectflam86',
 'Stinkypuss',
 'scottyshades',
 'Backer2004',
 'stoned2',
 'far333',
 'Redwood21',
 'billab914',
 'tewaris']

In [10]:
#which reviewer has the most reviews?

toprevs = Counter(beerDF.review_profilename)
toprevs.most_common(10)

[('northyorksammy', 5817),
 ('BuckeyeNation', 4661),
 ('mikesgroove', 4617),
 ('Thorpe429', 3518),
 ('womencantsail', 3497),
 ('NeroFiddled', 3488),
 ('ChainGangGuy', 3471),
 ('brentk56', 3357),
 ('Phyl21ca', 3179),
 ('WesWes', 3168)]

## Extracting Reviews

In [11]:
def get_beer_reviews(beer, common_users):
    mask = (beerDF.review_profilename.isin(common_users)) & (beerDF.beer_name==beer)
    reviews = beerDF[mask].sort('review_profilename')
    reviews = reviews[reviews.review_profilename.duplicated()==False]
    return reviews
beer_1_reviews = get_beer_reviews(beer_1, common_reviewers)
beer_2_reviews = get_beer_reviews(beer_2, common_reviewers)

cols = ['beer_name', 'review_profilename', 'review_overall', 'review_aroma', 'review_palate', 'review_taste']
beer_2_reviews[cols].head()

,beer_name,review_profilename,review_overall,review_aroma,review_palate,review_taste
595646,India Pale Ale,1fastz28,3.5,4.0,3.5,3.0
212302,India Pale Ale,49degrees,3.5,3.0,3.0,3.5
1348197,India Pale Ale,99bottles,4.5,4.0,4.0,4.5
1406683,India Pale Ale,9InchNails,3.5,3.5,4.0,3.5
595971,India Pale Ale,ADR,4.0,3.5,3.5,3.0


## Calculate Euclidean Distances

In [12]:
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import manhattan_distances
from scipy.stats.stats import pearsonr


ALL_FEATURES = ['review_overall', 'review_aroma', 'review_palate', 'review_taste']
def calculate_similarity(beer1, beer2):
    # find common reviewers
    beer_1_reviewers = beerDF[beerDF.beer_name==beer1].review_profilename.unique()
    beer_2_reviewers = beerDF[beerDF.beer_name==beer2].review_profilename.unique()
    common_reviewers = set(beer_1_reviewers).intersection(beer_2_reviewers)

    # get reviews
    beer_1_reviews = get_beer_reviews(beer1, common_reviewers)
    beer_2_reviews = get_beer_reviews(beer2, common_reviewers)
    dists = []
    for f in ALL_FEATURES:
        dists.append(euclidean_distances(beer_1_reviews[f], beer_2_reviews[f])[0][0])
    
    return dists

calculate_similarity(beer_1, beer_2)

[27.308423608842748, 27.37243138634199, 26.177280225416848, 29.828677476549309]

## Calculate the Similarity of a Set of Beers

In [13]:
# calculate only a subset for the demo
beers = ["Dale's Pale Ale", "Sierra Nevada Pale Ale", "Michelob Ultra",
         "Natural Light", "Bud Light", "Fat Tire Amber Ale", "Coors Light",
         "Blue Moon Belgian White", "60 Minute IPA", "Guinness Draught"]

# calculate everything for real production
# beers = df.beer_name.unique()

simple_distances = []
for beer1 in beers:
    print ("starting", beer1)
    for beer2 in beers:
        if beer1 != beer2:
            row = [beer1, beer2] + calculate_similarity(beer1, beer2)
            simple_distances.append(row)

starting Dale's Pale Ale
starting Sierra Nevada Pale Ale
starting Michelob Ultra
starting Natural Light
starting Bud Light
starting Fat Tire Amber Ale
starting Coors Light
starting Blue Moon Belgian White
starting 60 Minute IPA
starting Guinness Draught


## Inspect the Results

In [14]:
cols = ["beer1", "beer2", "overall_dist", "aroma_dist", "palate_dist", "taste_dist"]
simple_distances = pd.DataFrame(simple_distances, columns=cols)
simple_distances.tail()

,beer1,beer2,overall_dist,aroma_dist,palate_dist,taste_dist
85,Guinness Draught,Bud Light,43.854874,42.388088,45.694639,44.516851
86,Guinness Draught,Fat Tire Amber Ale,23.611438,21.095023,24.591665,22.632941
87,Guinness Draught,Coors Light,41.249242,38.337319,43.182751,40.416581
88,Guinness Draught,Blue Moon Belgian White,27.285527,22.169799,25.475478,23.780244
89,Guinness Draught,60 Minute IPA,31.200962,32.638168,32.190837,34.452866


## Allow Users to Customize the Weights

In [15]:
def calc_distance(dists, beer1, beer2, weights):
    mask = (dists.beer1==beer1) & (dists.beer2==beer2)
    row = dists[mask]
    row = row[['overall_dist', 'aroma_dist', 'palate_dist', 'taste_dist']]
    dist = weights * row
    return dist.sum(axis=1).tolist()[0]

weights = [2, 1, 1, 1] #Give your ratings here
print (calc_distance(simple_distances, "Fat Tire Amber Ale", "Dale's Pale Ale", weights))
print (calc_distance(simple_distances, "Fat Tire Amber Ale", "Michelob Ultra", weights))

86.40810158004888
153.0152903071523


## Recommend beers similar to ______________

In [16]:
my_beer = "60 Minute IPA"
results = []
for b in beers:
    if my_beer!=b:
        results.append((my_beer, b, calc_distance(simple_distances, my_beer, b, weights)))
sorted(results, key=lambda x: x[2])

[('60 Minute IPA', "Dale's Pale Ale", 86.14915895628911),
 ('60 Minute IPA', 'Sierra Nevada Pale Ale', 104.27352244928987),
 ('60 Minute IPA', 'Fat Tire Amber Ale', 113.30339542932701),
 ('60 Minute IPA', 'Blue Moon Belgian White', 131.96909004478798),
 ('60 Minute IPA', 'Guinness Draught', 161.68379444757014),
 ('60 Minute IPA', 'Natural Light', 168.19551166662126),
 ('60 Minute IPA', 'Michelob Ultra', 200.38047100332074),
 ('60 Minute IPA', 'Coors Light', 233.12536459010371),
 ('60 Minute IPA', 'Bud Light', 262.58747519219816)]

# Data Understanding 2

----


<span style="color:red">10 Points - Visualize the any important attributes appropriately. Important: Provide an interpretation for any charts or graphs.</span>

Insert Content Here!

# Modeling and Evaluation 1
---

<span style="color:red">10 Points - Train and adjust parameters</span>

LEFT OVER FROM LAST ASSIGNMENT - We utilized “ACCURACY” as our evaluation metric of choice because we want to classify a salary range and the diversity (race/national origin) category.  A low false positive score is ideal because we do not want to incorrectly classify the data points.  All things being equal, it would be better if we had more false negatives. For this dataset, logic steers us to be accurate overall since we have four classes of salary range and multiple classes for the diversity classification.  An argument can be made for looking at false negatives on a per class basis. For instance, after reviewing our initial data exploration, we found that the salary range class was unevenly distributed, so we might observe a low count of false positives. This makes sense since a person in a salary range with relatively few measured data points may trigger a low amount of false negatives under some models. 

# Modeling and Evaluation 2 - MUST USE PYTHON 2 KERNAL
---

<span style="color:red">10 Points - Evaluate and Compare</span>

Insert Content Here....

In [6]:
# This is a well known graphlab example that builds a recommendation system in 5 lines of code

import graphlab as gl

data = gl.SFrame.read_csv("beer_reviews.csv", 
                          column_type_hints={"review_overall":int})
data = data.dropna()
model = gl.recommender.create(data, user_id="review_profilename", item_id="beer_name", target="review_overall")
results = model.recommend(users=None, k=5)
model.save("my_model")

results.head() # the recommendation output

Finished parsing file C:\Users\natha_000\beer_reviews.csv

Parsing completed. Parsed 100 lines in 0.538085 secs.

Read 453422 lines. Lines per second: 675988

Finished parsing file C:\Users\natha_000\beer_reviews.csv

Parsing completed. Parsed 1586614 lines in 1.58794 secs.

...

recommendations finished on 29000/32909 queries. users per second: 1041.54

recommendations finished on 30000/32909 queries. users per second: 1026.35

recommendations finished on 31000/32909 queries. users per second: 1007.62

recommendations finished on 32000/32909 queries. users per second: 989.708

review_profilename,beer_name,score,rank
stcules,Bell's Hopslam Ale,3.94558264191,1
stcules,Founders KBS (KentuckyBreakfast Stout) ...,3.92761915053,2
stcules,World Wide Stout,3.90815318996,3
stcules,Double Bastard Ale,3.89729359622,4
stcules,Bourbon County BrandStout ...,3.88624387855,5
johnmichaelsen,Samuel Adams Boston Lager,3.86349514375,1
johnmichaelsen,60 Minute IPA,3.82922016705,2
johnmichaelsen,Brooklyn Black ChocolateStout ...,3.81866200889,3
johnmichaelsen,Dead Guy Ale,3.81344153235,4
johnmichaelsen,Bell's Oberon Ale,3.79559807727,5


# Modeling and Evaluation 3
---

<span style="color:red">10 Points - Summarize the Ramifications</span>

Insert Content Here!

# Modeling and Evaluation 4
---

<span style="color:red">20 Points - Summarize the Ramifications. </span>

Insert Content Here!

# Deployment
---


<span style="color:red">10 Points - Be critical of your performance and tell the reader how you current model might be usable by other parties. Did you achieve your goals? If not, can you reign in the utility of your modeling? How useful is your model for interested parties (i.e., the companies or organizations that might want to use it)? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.? </span>

LEFT OVER FROM LAST ASSIGNMENT - All Federal Agencies are required to use and store the data elements defined by Office of Personnel Management (OPM). The data elements are defined in the following OPM website: https://www.opm.gov/policy-data-oversight/pay-leave/salaries-wages/salary-tables/datadictionary.aspx

We strongly recommend all Human Resources and Workforce Planning offices of the United States Government, especially the Office of Personnel Management (OPM) will benefit from the use of the models and analysis contained in the document. 

# Exceptional Work
---

<span style="color:red">10 Points - You have free reign to provide additional analyses or combine analyses.</span>

# Appendix